In [3]:
!pip install langchain


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [10]:
pdf_path = "artifacts/source_documents"

from pathlib import Path

pdf_list = list(Path(pdf_path).absolute().rglob('*.pdf'))
len(pdf_list)

158

In [11]:
import glob
import os
from multiprocessing import Pool
from typing import List

from langchain.docstore.document import Document
from langchain.document_loaders import (
    CSVLoader,
    EverNoteLoader,
    PDFMinerLoader,
    TextLoader,
    UnstructuredEmailLoader,
    UnstructuredEPubLoader,
    UnstructuredHTMLLoader,
    UnstructuredMarkdownLoader,
    UnstructuredODTLoader,
    UnstructuredPowerPointLoader,
    UnstructuredWordDocumentLoader,
)
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from tqdm import tqdm

from configuration.config import CHROMA_SETTINGS
from configuration.config import app_config
from configuration.logging import logger

In [12]:
class MyElmLoader(UnstructuredEmailLoader):
    """Wrapper to fallback to text/plain when default does not work"""

    def load(self) -> List[Document]:
        """Wrapper adding fallback for elm without html"""
        try:
            try:
                doc = UnstructuredEmailLoader.load(self)
            except ValueError as e:
                if 'text/html content not found in email' in str(e):
                    # Try plain text
                    self.unstructured_kwargs["content_source"] = "text/plain"
                    doc = UnstructuredEmailLoader.load(self)
                else:
                    raise
        except Exception as e:
            # Add file_path to exception message
            raise type(e)(f"{self.file_path}: {e}") from e

        return doc

In [13]:

# Map file extensions to document loaders and their arguments
LOADER_MAPPING = {
    ".csv": (CSVLoader, {}),
    # ".docx": (Docx2txtLoader, {}),
    ".doc": (UnstructuredWordDocumentLoader, {}),
    ".docx": (UnstructuredWordDocumentLoader, {}),
    ".enex": (EverNoteLoader, {}),
    ".eml": (MyElmLoader, {}),
    ".epub": (UnstructuredEPubLoader, {}),
    ".html": (UnstructuredHTMLLoader, {}),
    ".md": (UnstructuredMarkdownLoader, {}),
    ".odt": (UnstructuredODTLoader, {}),
    ".pdf": (PDFMinerLoader, {}),
    ".ppt": (UnstructuredPowerPointLoader, {}),
    ".pptx": (UnstructuredPowerPointLoader, {}),
    ".txt": (TextLoader, {"encoding": "utf8"}),
    # Add more mappings for other file extensions and loaders as needed
}


In [17]:
def load_single_document(file_path: str) -> Document:
    ext = "." + file_path.rsplit(".", 1)[-1]
    try:
        if ext in LOADER_MAPPING:
            loader_class, loader_args = LOADER_MAPPING[ext]
            loader = loader_class(file_path, **loader_args)
            return loader.load()[0]

        raise ValueError(f"Unsupported file extension '{ext}'")
    except Exception as error:
        logger.error(f"Error at : `{file_path}` : `{str(error)}`")
        
# docs = [load_single_document(str(f)) for f in pdf_list]

2023-06-21 13:43:40.080 | ERROR    | __main__:load_single_document:11 - Error at : `/home/khaled-adrani/Documents/ELGEN/elgen-esg-conversational-bot/artifacts/source_documents/Royal Swinkels Family Breweries/0.pdf` : `No /Root object! - Is this really a PDF?`
2023-06-21 13:43:40.084 | ERROR    | __main__:load_single_document:11 - Error at : `/home/khaled-adrani/Documents/ELGEN/elgen-esg-conversational-bot/artifacts/source_documents/Royal Swinkels Family Breweries/1.pdf` : `No /Root object! - Is this really a PDF?`
2023-06-21 13:44:40.240 | ERROR    | __main__:load_single_document:11 - Error at : `/home/khaled-adrani/Documents/ELGEN/elgen-esg-conversational-bot/artifacts/source_documents/Henri Willig Groep/3.pdf` : `No /Root object! - Is this really a PDF?`
2023-06-21 13:44:40.242 | ERROR    | __main__:load_single_document:11 - Error at : `/home/khaled-adrani/Documents/ELGEN/elgen-esg-conversational-bot/artifacts/source_documents/Henri Willig Groep/2.pdf` : `No /Root object! - Is this r

The PDF <_io.BufferedReader name='/home/khaled-adrani/Documents/ELGEN/elgen-esg-conversational-bot/artifacts/source_documents/FujiFilm/0.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


2023-06-21 13:54:53.147 | ERROR    | __main__:load_single_document:11 - Error at : `/home/khaled-adrani/Documents/ELGEN/elgen-esg-conversational-bot/artifacts/source_documents/KPN NV/10.pdf` : `No /Root object! - Is this really a PDF?`
2023-06-21 13:54:53.162 | ERROR    | __main__:load_single_document:11 - Error at : `/home/khaled-adrani/Documents/ELGEN/elgen-esg-conversational-bot/artifacts/source_documents/KPN NV/11.pdf` : `No /Root object! - Is this really a PDF?`
2023-06-21 13:54:53.176 | ERROR    | __main__:load_single_document:11 - Error at : `/home/khaled-adrani/Documents/ELGEN/elgen-esg-conversational-bot/artifacts/source_documents/KPN NV/7.pdf` : `No /Root object! - Is this really a PDF?`
2023-06-21 13:55:33.057 | ERROR    | __main__:load_single_document:11 - Error at : `/home/khaled-adrani/Documents/ELGEN/elgen-esg-conversational-bot/artifacts/source_documents/KPN NV/9.pdf` : `No /Root object! - Is this really a PDF?`
2023-06-21 13:56:07.960 | ERROR    | __main__:load_single_

The PDF <_io.BufferedReader name='/home/khaled-adrani/Documents/ELGEN/elgen-esg-conversational-bot/artifacts/source_documents/Royal Philips NV/4.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


2023-06-21 14:18:38.818 | ERROR    | __main__:load_single_document:11 - Error at : `/home/khaled-adrani/Documents/ELGEN/elgen-esg-conversational-bot/artifacts/source_documents/Royal Philips NV/6.pdf` : `No /Root object! - Is this really a PDF?`
2023-06-21 14:21:30.326 | ERROR    | __main__:load_single_document:11 - Error at : `/home/khaled-adrani/Documents/ELGEN/elgen-esg-conversational-bot/artifacts/source_documents/NN Group/7.pdf` : `No /Root object! - Is this really a PDF?`
2023-06-21 14:22:31.900 | ERROR    | __main__:load_single_document:11 - Error at : `/home/khaled-adrani/Documents/ELGEN/elgen-esg-conversational-bot/artifacts/source_documents/NN Group/6.pdf` : `No /Root object! - Is this really a PDF?`
2023-06-21 14:24:03.308 | ERROR    | __main__:load_single_document:11 - Error at : `/home/khaled-adrani/Documents/ELGEN/elgen-esg-conversational-bot/artifacts/source_documents/VolkerWessels/2.pdf` : `No /Root object! - Is this really a PDF?`
2023-06-21 14:26:07.084 | ERROR    | __

In [ ]:
def load_documents(source_dir: str, ignored_files: List[str] = []) -> List[Document]:
    """
    Loads all documents from the source documents directory, ignoring specified files
    """
    all_files = []
    for ext in LOADER_MAPPING:
        all_files.extend(
            glob.glob(os.path.join(source_dir, f"**/*{ext}"), recursive=True)
        )
    filtered_files = [file_path for file_path in all_files if file_path not in ignored_files]

    with Pool(processes=os.cpu_count()) as pool:
        results = []
        with tqdm(total=len(filtered_files), desc='Loading new documents', ncols=80) as pbar:
            for doc in pool.imap_unordered(load_single_document, filtered_files):
                results.append(doc)
                pbar.update()

    print("len before filter ", len(results))

    results = [res for res in results if res is not None]

    print("len after filter  ", len(results))

    return results

#docs = load_documents(pdf_path)

In [18]:
doc = docs[0]
str(doc)

'page_content=\'EYE CARE, WE CARE MOREAnnual Report 2020\\x0cOUR OPERATIONSOUR PEOPLEOUR FINANCIAL RESULTSPROJECTED MARKET GROWTH BY CATEGORY*(FULL YEAR, 3-YEAR CAGR)GrandVision is a leading global optical retailer with more than 7,200 stores worldwide and a growing online presence. We offer our customers expert eyecare services along with a large selection of unique and stylish prescription eyeglasses, sunglasses, contact lenses and eyecare products.7,26039K+2.8%68%2.0%€266M14.1%434K+23.03.3%€3.5BDaily store visits Eyeglasses and sunglasses soldFYFYFYFYFYFYYEYEYEStores RevenueEmployees(headcount) Training hoursper FTEAdjusted EBITATotal female FTEsComparabledecrease* Source: Euromonitor for 2020-2024Please note: FY = Full-year; YE = Year-end15.1MOpticalSunglassesContact lensGrandVision: 2020 by the numbers\\x0cOur five strategic priorities0105020304Drive comparable revenue growthAchieve lead position in all markets where we operateLeverage global economies of scale and skillAccelerate

In [20]:
corpus = "<end> ".join(str(d) for d in docs)

In [ ]:
from typing import Tuple 
from nltk import ngrams
import random
import sys
import time

class TripleGramLLM:
    def __init__(self,corpus:str):
        self.corpus = corpus
        self.model = {}
        self.tokenized = self.corpus.lower().split()
        
        
    def __len__(self):
        return len(self.model)
        
    def train(self):
        model = self.model
                
        ngram_triplets = ngrams(self.tokenized, 3)

        for word1, word2, word3 in ngram_triplets:
            key = (word1, word2)
            if key in model:
                model[key].append(word3)
            else:
                model[key] = [word3]

        return model
    
    def get_semantic_best_choice(self, key, seed):
        values = model[key]
        query = " ".join(seed)
        
        target = compute_avg_weight(query, embed)
        target = target.reshape(1, -1)
        
        values_embeddings = []
        
        for value in values:
            v = get_embeddings(value, embed)
            v = v.reshape(1,-1)
            
            similarity_score = cosine_similarity(target, v)
            values_embeddings.append(similarity_score)
            
        current_score = -1
        max_index = -1
        for i in range(1,values):
            if values[i]>current_score:
                max_index = i
                
        return values[i]
            
        
    
    def _generate_from_seed(self, seed : Tuple[str,str], length:int = 20):
        seed = [e.lower() for e in seed]
        model = self.model
        current_word = seed[1]
        prev_word = seed[0]
        generated_text = [prev_word, current_word]

        for _ in range(length - 1):
            key = (prev_word, current_word)
            if key in model:
                next_word = random.choice(model[key])
                generated_text.append(next_word)
                prev_word = current_word
                current_word = next_word
            else:
                next_word = random.choice(list[model.values()])
                generated_text.append(next_word)
                prev_word = current_word
                current_word = next_word
        return generated_text
                
    def generate_from_seed(self, seed : Tuple[str,str], length:int = 20, streaming=False):
        text = ' '.join(self._generate_from_seed(seed=seed, length=length))
        
        if streaming:
            for char in text:

                sys.stdout.write(char)
                sys.stdout.flush()
                time.sleep(0.02)  # Optional delay for demonstration purposes
        
        return text
    
    def save_model(self):
        with open('triple_gram_model.pkl','wb') as f:
            pickle.dump(llm, f)
            
    @staticmethod
    def load_model():
        with open('triple_gram_model.pkl','rb') as f:
            return pickle.load(f) 
        
        
llm = TripleGramLLM(corpus=corpus)
llm.train()
len(llm)

In [ ]:
from pprint import pprint
text = llm.generate_from_seed(('I', 'handle'), 100)
text